In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
from PIL import Image
import imageio
import os
import csv

# Denoising and Detail Transfer

In [2]:
def GaussianRange(r):
    GaussianRange = np.zeros(256)
    part = (1 / math.sqrt(2*math.pi* (r**2)))
    for i in range(256):
        GaussianRange[i] = part * np.exp(-(i**2)/(2 * (r**2)))  
    return GaussianRange

In [3]:
def GaussianSpatial(d):
    part = (1 / math.sqrt(2*math.pi* (d**2)))
    GaussianSpatial = np.zeros((2 * d + 1,2 * d + 1))
    range_d = -d + np.arange(2 * d+ 1)
    for i in range_d:
        for j in range_d:
            GaussianSpatial[i+d][j+d]= part * np.exp(-(i **2 + j **2) /(2 * (d**2) ) )
    return GaussianSpatial

In [4]:
def jointBilateralFilter(no_flash, flash, d, r):
    x, y, rgb = no_flash.shape
    noflashImg =np.pad(no_flash, ((d, d), (d, d), (0, 0)), 'symmetric') 
    flashImg = np.pad(flash, ((d, d), (d, d), (0, 0)), 'symmetric')
    outputImg = np.zeros((x,y,rgb), np.uint8)
    GaussianR = GaussianRange(r)
    GaussianS = GaussianSpatial(d)
    
    for i in range(d, d + x):
        for j in range(d, d + y):
            kp = 0
            sum1 = 0
            for k in range (rgb):
                pInten = flashImg[i][j][k]
                qInten = flashImg[i-d : i+d+1 , j-d : j+d+1, k] 
                Rangepad = GaussianR[abs(qInten - pInten)]
                Rangepad2 = Rangepad * noflashImg[i-d : i+d+1 , j-d : j+d+1, k] 
                mul=Rangepad*GaussianS
                kp= np.sum(mul)
                mul2= Rangepad2*GaussianS
                sum1 =  np.sum(mul2)
                outputImg [i-d][j-d][k]= (1/kp)*sum1
                
    return outputImg

def DetailTransfer(flash, e, F_base, denoiseImg):
    h, w, rgb = flash.shape
    F_detail = np.zeros((h,w,rgb), np.uint8)
    for x in range(h):
        for y in range(w):
            for i in range (rgb):
                F_detail[x][y][i] = (flash[x][y][i] + e)/(F_base[x][y][i] + e) * denoiseImg[x][y][i]
                

    return F_detail
                        

# Detecting shadow

In [5]:
def flash_no_flash_process(k, a, f, d, fast, use_hist):
    _, ext = os.path.splitext(a)
    # Load images
    A = cv2.imread('./Test_Photos/'+d+'/'+a)
    F = cv2.imread('./Test_Photos/'+d+'/'+f)
    
    if A.shape[0] * A.shape[1]/1000000 < 1:
        sigmaSpace = 3
    elif A.shape[0] * A.shape[1]/1000000 < 2:
        sigmaSpace = 9
    else:
        sigmaSpace = 36
        
    sigmaColor = (255 * 0.1)
    
    # Bilateral filter on ambient image
    if fast: A_base = cv2.bilateralFilter(A,sigmaSpace,sigmaColor,sigmaSpace)
    else: A_base = jointBilateralFilter(A,A,sigmaSpace,sigmaColor)
    
    # Joint bilateral filter with flash image as the guide
    if fast: A_nr = cv2.ximgproc.jointBilateralFilter(F, A, sigmaSpace,(255 * 0.001),sigmaSpace)
    else: A_nr = jointBilateralFilter(A, F, sigmaSpace,(255 * 0.001))
    cv2.imwrite('./Output/'+d+'/'+k+'_ambient_joint'+ext, A_nr)
    
    # Extract detail of flash image
    epsilon = np.ones(F.shape, dtype = np.float32) * 0.02
    if fast: F_base = cv2.bilateralFilter(F,sigmaSpace, 50,sigmaSpace)
    else: F_base = jointBilateralFilter(F,F,sigmaSpace,50)
    F_detail = cv2.divide(F.astype(np.float32) + epsilon, F_base.astype(np.float32) + epsilon)
    F_detail = np.clip(F_detail,0,2)
    A_nrdetail = cv2.multiply(F_detail, A_nr.astype(np.float32))
    A_nrdetail = np.clip(A_nrdetail, 0, 255).astype(np.uint8)
    cv2.imwrite('./Output/'+d+'/'+k+'_ambient_nrdetail'+ext, A_nrdetail)
        
    # Heuristic of shadow and specularity detection
    A_grey = cv2.cvtColor(A, cv2.COLOR_BGR2GRAY)
    A_grey = cv2.GaussianBlur(A_grey,(5,5),0)
    A_grey = cv2.normalize(A_grey, None, 0, 255, cv2.NORM_MINMAX)
    F_grey = cv2.cvtColor(F, cv2.COLOR_BGR2GRAY)
    F_grey = cv2.GaussianBlur(F_grey,(5,5),0)
    F_grey = cv2.normalize(F_grey, None, 0, 255, cv2.NORM_MINMAX)
    
    A_shadIndex = 20
    F_shadIndex = 20
    if use_hist:
        A_hist = cv2.calcHist(A_grey,[0],None,[256],[0,256])
        A_hist = np.cumsum(A_hist)
        A_shadIndex = np.searchsorted(A_hist, round(A_hist[-1]*0.05), side='right')

        F_hist = cv2.calcHist(F_grey,[0],None,[256],[0,256])
        F_hist = np.cumsum(F_hist)
        F_shadIndex = np.searchsorted(F_hist, round(F_hist[-1]*0.2), side='right')
    F_specIndex = 245
    

    _, F_shadow = cv2.threshold(F_grey, F_shadIndex, 255, cv2.THRESH_BINARY_INV)
    _, A_shadow = cv2.threshold(A_grey, A_shadIndex, 255, cv2.THRESH_BINARY_INV)
    
    kernel =  cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    F_shadow = cv2.dilate(F_shadow,kernel,iterations = 1)
    A_shadow = cv2.erode(A_shadow,kernel,iterations = 1)
    
    M_shadow = cv2.bitwise_and(F_shadow, cv2.bitwise_not(A_shadow))
    M_shadow = M_shadow.astype(np.uint8)
    M_shadow = cv2.dilate(M_shadow,kernel,iterations = 1)
    
    _, F_specular = cv2.threshold(F_grey, F_specIndex, 255, cv2.THRESH_BINARY)
    F_specular = cv2.dilate(F_specular,kernel,iterations = 1)
    
    M_art = cv2.bitwise_or(M_shadow, F_specular)
    cv2.imwrite('./Output/'+d+'/'+k+'_mask_artifact'+ext, M_art)
        
    # Save output image
    A_final = np.zeros(A.shape, dtype = np.uint8)
    for i in range(3):
        A_final[:,:,i] = cv2.bitwise_or(cv2.bitwise_and(A_nrdetail[:,:,i],cv2.bitwise_not(M_art)), cv2.bitwise_and(A_base[:,:,i],M_art))
    cv2.imwrite('./Output/'+d+'/'+k+'_ambient_final'+ext,  A_final)

In [6]:
if not os.path.exists('./Output'):
    os.mkdir('./Output')
directories = os.listdir('Test_Photos')
for d in directories:
    if d[0] == '.': continue
    print(d)
    if not os.path.exists('./Output/'+d):
        os.mkdir('./Output/'+d)
    if d == "test":
        flash_no_flash_process("test", 'cave01_01_noflash.jpg', 'cave01_00_flash.jpg', "test", False, False)
    else:
        with open('./Test_Photos/'+d+'/meta.csv', 'r') as csvfile:
            r = csv.DictReader(csvfile)
            for row in r:
                flash_no_flash_process(row['key'], row['ambient'], row['flash'], d, True, True)


Object_Photos
people
plants
Rooms_Photos
shelves
test
Toys_Photos
